In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import cycle, islice
from os import listdir, path
from os.path import isfile, join
from PIL import Image
import numpy as np
import math
from datetime import datetime
import tensorflow as tf

In [6]:
def red(image):
    arr = np.asarray(image).copy()
    rgb = arr[:,:,1:3]
    arr[:,:,1:3][np.where(rgb == 255)] = 0
    
    return Image.fromarray(arr)


def black(image):
    arr = np.asarray(image).copy()
    rgb = arr[:,:,0:3]
    arr[:,:,0:3][np.where(rgb == 255)] = 0
    
    return Image.fromarray(arr)


def load_rank_samples(rank_images_dir):
    for name in listdir(rank_images_dir):
        full_path = join(rank_images_dir, name)
        if not isfile(full_path):
            continue

        rank = path.splitext(path.basename(full_path))[0]
        image = Image.open(full_path)
        red_image = red(image)
        black_image = black(image)

        yield (red_image, rank)
        yield (black_image, rank)


def warp(img, angle, center, new_center, scale):
    angle = -angle/180.0*math.pi
    nx,ny = x,y = center
    sx=sy=1.0
    (nx,ny) = new_center
    (sx,sy) = scale
    cosine = math.cos(angle)
    sine = math.sin(angle)
    a = cosine/sx
    b = sine/sx
    c = x-nx*a-ny*b
    d = -sine/sy
    e = cosine/sy
    f = y-nx*d-ny*e
    
    fff = Image.new('RGBA', img.size, (255,)*4)
    warped_img = img.transform(img.size, Image.AFFINE, (a,b,c,d,e,f))
    img = Image.composite(warped_img, fff, warped_img)
    
    return img.convert('L')


def warp_rnd(image):
    r = lambda v: v * (.5 - np.random.rand())
    width, height = image.size
    
    angle = r(20)
    center = width/2 + r(width/10), height/2 + r(height/10)
    new_center = width/2 + r(width/20), height/2 + r(height/20)
    scale = 1 + r(.3), 1 + r(.3)
    
    return warp(image, angle, center, new_center, scale)


def gen_rank_samples(rank_images_dir, count, warping=True):
    samples = list(load_rank_samples(rank_images_dir))
    if warping:
        samples = [(warp_rnd(image), label) for (image, label) in islice(cycle(samples), 0, count)]
    images, labels = zip(*samples)
    vocabulary = sorted(list(set(labels)))
    labels = [vocabulary.index(label) for label in labels]
    return images, labels, vocabulary

In [58]:
images, labels, vocabulary = gen_rank_samples('res/cards/rank', 10)
idx = np.asarray(labels)
l = np.zeros((len(labels), len(vocabulary)), dtype=np.float32)
l[np.arange(len(labels)), idx] = 1
l.dtype

dtype('float32')

In [75]:
def card_rank_model_fn(features, labels, mode):
    x = tf.placeholder(tf.float32, [None, 520])
    W = tf.Variable(tf.zeros([520, 13]))
    b = tf.Variable(tf.zeros([13]))
    y = tf.nn.softmax(tf.matmul(features['image'], W) + b)
    
    y_target = tf.placeholder(tf.float32, [None, 13])
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(y), reduction_indices=[1]))
    
    optimizer = tf.train.GradientDescentOptimizer(0.005)
    train_step = optimizer.minimize(cross_entropy)

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=cross_entropy,
        train_op=train_step
    )

def card_rank_train_input_fn(steps, batch_count, batch_size):
    count = steps * batch_count * batch_size
    images, labels, vocabulary = gen_rank_samples('res/cards/rank', count)
    asarray = lambda img: np.asarray(img.getdata(), dtype=np.float32)
    features = {
        'image': np.vstack([asarray(img) for img in images])
    }
    
    idx = np.asarray(labels)
    labels = np.zeros((len(labels), len(vocabulary)), dtype=np.float32)
    labels[np.arange(len(labels)), idx] = 1
    
    print(labels)
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()

def card_rank_eval_input_fn():
    (images, labels) = load_rank_images('res/cards/rank')
    asarray = lambda img: np.asarray(img.getdata(), dtype=np.float32)
    features = {
        'image': np.vstack([asarray(img) for img in images])
    }
    
    idx = np.asarray(labels)
    labels = np.zeros((len(labels), len(vocabulary)), dtype=np.float32)
    labels[np.arange(len(labels)), idx] = 1
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    return dataset.make_one_shot_iterator().get_next()

In [76]:
classifier = tf.estimator.Estimator(
    model_fn = card_rank_model_fn,
    model_dir = 'models/card_rank'
)

train_steps = 5
train_batch_count = 10
train_batch_size = 10

classifier.train(
    input_fn = lambda: card_rank_train_input_fn(train_steps, train_batch_count, train_batch_size),
    steps = train_steps
)

metrics = classifier.evaluate(
    input_fn = card_rank_eval_input_fn
)

print('Accuracy: {accuracy:0.3f}'.format(**metrics))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_master': '', '_save_checkpoints_steps': None, '_tf_random_seed': None, '_is_chief': True, '_service': None, '_model_dir': 'models/card_rank', '_task_id': 0, '_session_config': None, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69f0d0f128>, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_save_summary_steps': 100}
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/card_rank/model.ckpt-0
INFO:tensorflow:Saving checkpoints for 0 into models/card_rank/model.ckpt.
ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: NaN loss during training.